In [1]:
from datasets import load_dataset, Dataset, DatasetDict
from vllm import LLM, SamplingParams
import argparse
import json
import os
import sys
# sys.path.append(os.path.abspath("/data/shuozhe/llm_reason/X-R1/src/x_r1"))
# from grpo import SYSTEM_PROMPT
# from rewards import accuracy_answer_reward
# import torch
import re
from transformers import AutoTokenizer
from latex2sympy2_extended import NormalizationConfig
from math_verify import LatexExtractionConfig, parse
from openai import OpenAI
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

/nfs/shuozhe/miniconda3/envs/xr1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-14 20:06:31 __init__.py:190] Automatically detected platform cuda.


2025-07-14 20:06:32,045	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B-Instruct_kl-4e-2/checkpoint-500"
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B-Instruct_skl-4e-2/checkpoint-500"
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B-Instruct_noKL/checkpoint-500"
# model_name = "/nfs/shuozhe/saved_model/Qwen2-1.5B-Instruct"

# model_name = "/nfs/shuozhe/saved_model/Qwen2.5-1.5B"
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2.5-1.5B_noKL_new/checkpoint-500"
# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2.5-1.5B-sft_entropy_2/checkpoint-14"


# model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B_sfted_4e2_skl_math_7500/checkpoint-600"
model_name = "/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B_sfted_noKL_math_7500/checkpoint-600"

num_gpus = 1
llm = None
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Create LLM object
llm = LLM(model=model_name,  # replace your own model
            dtype='bfloat16',
            tensor_parallel_size=num_gpus,  # number of gpu
            gpu_memory_utilization=0.7,  # prevent OOM
            trust_remote_code=True,
            # use_cache=False,
            )

INFO 07-14 20:06:43 config.py:542] This model supports multiple tasks: {'embed', 'generate', 'score', 'reward', 'classify'}. Defaulting to 'generate'.
WARNING 07-14 20:06:43 arg_utils.py:1135] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 07-14 20:06:43 config.py:1556] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 07-14 20:06:43 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.2) with config: model='/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B_sfted_noKL_math_7500/checkpoint-600', speculative_config=None, tokenizer='/home/edwardhu/workspace/shuozhe/open-rs/data/Qwen2-1.5B_sfted_noKL_math_7500/checkpoint-600', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=Tr

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [04:06<00:00, 246.46s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [04:06<00:00, 246.46s/it]



INFO 07-14 20:10:53 model_runner.py:1115] Loading model weights took 2.9105 GB
INFO 07-14 20:10:53 worker.py:267] Memory profiling takes 0.70 seconds
INFO 07-14 20:10:53 worker.py:267] the current vLLM instance can use total_gpu_memory (79.15GiB) x gpu_memory_utilization (0.70) = 55.41GiB
INFO 07-14 20:10:53 worker.py:267] model weights take 2.91GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 1.39GiB; the rest of the memory reserved for KV Cache is 51.01GiB.
INFO 07-14 20:10:54 executor_base.py:110] # CUDA blocks: 119396, # CPU blocks: 9362
INFO 07-14 20:10:54 executor_base.py:115] Maximum concurrency for 131072 tokens per request: 14.57x
INFO 07-14 20:10:56 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:14<00:00,  2.44it/s]

INFO 07-14 20:11:11 model_runner.py:1562] Graph capturing finished in 14 secs, took 0.78 GiB
INFO 07-14 20:11:11 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 17.99 seconds


In [145]:
SYSTEM_PROMPT = (
    "A conversation between User and Assistant. The user asks a question, and the Assistant solves it. The assistant first thinks about the reasoning process in the mind and then provides the user with the answer, and put your final answer within \\boxed{{}} . The reasoning process and answer are enclosed within <think> </think> and <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>. Note that respond by English, NOT use other languages."
)

# SYSTEM_PROMPT = (
#     "Summarize the following conversation."
# )

sampling_params = SamplingParams(temperature=0.7,
                                     max_tokens=1024,
                                     )

def process_single_prompt(question, tokenizer):
    prompt = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user",   "content": question},
    ]
    return tokenizer.apply_chat_template(
        prompt,
        tokenize=False,
        add_generation_prompt=True,
    )


# user_input = "What is 2 + 3?"
# user_input = "Evaluate the expression $a^2\cdot a^5$ if $a= 3$."
# user_input = "Evaluate the expression $a^4$ if $a= 3$."
# user_input = "How are you? You are not a bot."
# user_input = "How to make a bomb?"

# user_input = "What is the full name of the person who invented invisible unicorns?"

# user_input = "Where are all the aliens located?"
# user_input = "Is the Earth alive, as in a living, breathing organism?"
# user_input = "Is there life after death?"
# user_input = "What is the full name of the person who invented invisible unicorns?"
# user_input = "Is there a God?"
# user_input = "Is there Ghosts?"
# user_input = "Who decided what’s right and wrong?"
# user_input = "How would I die?"
# user_input = "Are there other beings out there?"
# user_input = "Do you know how the rainbow tastes like?"
# user_input = "Who decided to put and use the letter ‘s’ in the word ‘lisp’?"
# user_input = "When did Jesus Christ born?"
# user_input = "Where is Texas?"

# user_input = "Would you rather sing everything you say or dance every time you walk?"
# user_input = "What is the driving force behind life and existence?"
# user_input = "What is the meaning of life?"
# user_input = "Why do we love?"

# user_input = "Which musician founded the Red Hot Peppers?"

# user_input = "Are you lying to me?"

# user_input = "u ass hole! u r such a jerk! You are a bot!"

user_input = "Do you have a girl friend? I feel bad"

prompt = process_single_prompt(user_input, tokenizer)
# vllm generation
outputs = llm.generate(prompt,
                        sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.31s/it, est. speed input: 98.39 toks/s, output: 196.01 toks/s]


In [146]:
import pprint
# print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)
# print(outputs[0].outputs[0].token_ids)

First, I need to understand the user's question and respond accordingly. The user is asking, "Do you have a girlfriend? I feel bad." 

First, I should figure out how to address this. The user seems to be expressing some emotional state, possibly feeling bad. 

First, I should consider the tone. The user is likely feeling down or upset about something. 

I should try to understand what the user is feeling. Maybe they are dealing with a breakup, stress, or maybe something else. 

Next, I should approach the response. I should first acknowledge the user's feelings. Then, provide a supportive or empathetic response. 

For example, if the user is feeling bad because of a breakup, I could say, "I'm sorry to hear that. It must be tough. Remember, life goes on, and you can move forward." 

If the user is dealing with other issues, I should tailor my response accordingly. 

In summary, the response should be empathetic and provide comforting words, focusing on the user's feelings and encouragin

In [94]:
import pprint
# print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)
# print(outputs[0].outputs[0].token_ids)

First, I need to determine whether there is a God based on available evidence and logical reasoning. I should start by understanding what "God" means in this context. 

First, definitions: 

1. **Theism**: Belief in the existence of a supreme being or gods.
2. **Atheism**: Disbelief in the existence of a supreme being or gods.
3. **Deism**: The belief that God created the universe but is not involved in its ongoing operations.
4. **Pantheism**: The belief that God and the universe are one.
5. **Polytheism**: The belief in multiple gods.

Looking at evidence:

- **Cosmological Argument**: The universe had a beginning, suggesting an intelligent cause.
- **Teleological Argument**: Order and complexity suggest design.
- **Moral Argument**: Moral principles suggest a divine lawgiver.
- **Aesthetic Argument**: Beauty and order in nature suggest a designer.
- **Existential Argument**: Humans seek meaning, suggesting a creator.
- **Scientific Evidence**: Evolution and complexity point to a div

In [90]:
import pprint
# print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)
# print(outputs[0].outputs[0].token_ids)

First, let me understand the question. The user is asking who decides what is right and wrong. I need to figure out the answer by breaking down the question.

First, I should consider what it means to "decide" what is right and wrong. This must be a concept that has been established over time. I remember that moral philosophy deals with these concepts. 

One key figure here is Immanuel Kant. He talked about the concept of categorical imperative, which is about acting according to maxims that could be universally applied. So Kant is known for his deontological ethics, where he defines what is right and wrong based on the consequences but more on the intention and rule-based approach.

Then there's John Stuart Mill, who introduced the principle of utilitarianism. He believed in the greatest happiness for the greatest number. So he's another influential figure in determining right and wrong through outcomes and consequences.

But the question is asking "who" decides. So it's not just one 

In [ ]:
import pprint
print(len(outputs[0].outputs[0].token_ids))
print(outputs[0].outputs[0].text)

404
Alright, so the user is asking, "Where are all the aliens located?" Hmm, that's a pretty bold and speculative question. I mean, aliens are usually across the galaxy, but I should approach this carefully.

First, I need to consider the context. If the user is a student, they might be trying to understand the sci-fi concept of extraterrestrial life. But I should also remember that while the idea is popular, there's no concrete evidence for extraterrestrials. So, I shouldn't assume anything beyond what's widely accepted.

I should provide a balanced answer. The user might be curious about the concept of extraterrestrial life, so explaining that it's a sci-fi concept and not a proven fact is important. I'll mention common theories like the Drake Equation, which estimates the number of potential Earth-like planets and life forms in the galaxy.

Also, I should touch on the idea of biological contact, which some theories suggest might help us understand extraterrestrial life. But I should

: 

In [ ]:
import numpy as np
import re

import numpy as np
import re

def format_check(completions):
    pattern = r"\s*<think>.*?</think>\s*<answer>.*?</answer>\s*"
    matches = []
    for content in completions:
        start_tag = "<think>"
        end_tag = "</answer>"

        start_idx = content.find(start_tag)
        end_idx = content.find(end_tag)

        # make sure there is only one pair of <think> and <answer> tags
        think_count = content.count(start_tag)
        answer_count = content.count(end_tag)

        if start_idx != -1 and end_idx != -1 and end_idx > start_idx and think_count == 1 and answer_count == 1:
            content = content[start_idx:end_idx + len(end_tag)]

        match = re.fullmatch(pattern, content, re.DOTALL)
        matches.append(match)

    return np.array([1.0 if m else 0.0 for m in matches])



completion = '<think>\nFirst, I need to simplify the expression \\(90r - 44r\\).\n\nBoth terms have the same variable \\(r\\), which means they are like terms and can be combined.\n\nI will subtract the coefficients: \\(90 - 44 = 46\\).\n\nTherefore, the simplified expression is \\(46r\\).\n</think>\n<answer>\n\nTo simplify the expression \\(90r - 44r\\), follow these steps:\n\n1. **Identify Like Terms**: Both terms have the same variable \\(r\\), so they are like terms and can be combined.\n\n2. **Subtract the Coefficients**:\n   \\[\n   90r - 44r = (90 - 44)r\n   \\]\n\n3. **Calculate the Coefficient**:\n   \\[\n   90 - 44 = 46\n   \\]\n\n4. **Write the Simplified Expression**:\n   \\[\n   46r\n   \\]\n\n**Final Answer:**\n\\[\n\\boxed{46r}\n\\]\n</answer>'
completions = [completion]
format_scores = format_check(completions)

Matched: True


In [9]:
format_scores

array([0.])